In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: selenium in c:\users\dohai\appdata\local\programs\python\python312\lib\site-packages (4.22.0)




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# wehn we run the code remotly
!pip install --upgrade selenium
!pip install webdriver_manager
!apt-get update
!apt install chromium-chromedriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # Import Options class
# Configure Chrome options
chrome_options = Options()
# Run Chrome in headless mode
chrome_options.add_argument('--headless=new') # change headless mode to 'new'
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Overcome limited resource problems
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1280x1696')
#chrome_options.add_argument('--single-process') # Removed to address potential conflicts
#chrome_options.add_argument('--disable-dev-tools') # Removed for potential stability
#chrome_options.add_argument('--no-zygote')  # Removed for potential conflicts
chrome_options.add_argument(f'--user-data-dir=/tmp/chrome_user_data')
chrome_options.add_argument('--remote-debugging-port=9222')
# Add arguments to specifically address the crash
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-background-networking")
chrome_options.add_argument("--safebrowsing-disable-auto-update")
chrome_options.add_argument("--disable-sync")
chrome_options.add_argument("--disable-translate")


# Define the path to the Chromium browser explicitly
chrome_options.binary_location = '/usr/bin/chromium-browser'  # or /usr/bin/google-chrome for regular Chrome


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [36]:

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException  # Add this import

In [37]:
def scroll_until_new_items(driver, timeout=10):
    # Find current count of perfume links
    initial_count = len(driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]"))
    
    # Scroll to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for more elements to appear
    WebDriverWait(driver, timeout).until(
        lambda driver: len(driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")) > initial_count
    )

In [39]:
# Define a function to scroll and wait for new items
def scroll_until_new_items(driver, timeout=10):
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Small delay to let new items load

    # Wait until the page height changes (indicating new items loaded)
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script("return document.body.scrollHeight") > last_height
        )
    except TimeoutException:
        print("No more items to load or page height did not change.")


In [40]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import re  # Import regular expressions

# Initialize the WebDriver when we run the code locally
driver = webdriver.Chrome()

# when we run the code remotly
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the provided URL
driver.get("https://www.goldenscent.com/c/new.html?action=cat&id=243")

# Wait for the page to load
time.sleep(5)  # Adjust the wait time if needed

# Initialize an empty list to store unique hrefs
unique_hrefs = []

previous_length = 0
scroll_attempts = 0

# Loop to collect unique hrefs for up to 200 perfumes
while len(unique_hrefs) < 20:

    print(len(unique_hrefs))
    # Locate the perfume links and extract their href attributes
    golden_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    golden_hrefs = [link.get_attribute('href') for link in golden_links]
   

   
    # Collect unique hrefs
    for href in golden_hrefs:
        if href not in unique_hrefs:
            unique_hrefs.append(href)
            
        if len(unique_hrefs) >= 200:  # Stop once we have 200 unique hrefs
            break

   # Check if new links were added; if not, increase scroll attempts
    if len(unique_hrefs) == previous_length:
        scroll_attempts += 1
    else:
        scroll_attempts = 0  # Reset if new items are found

    previous_length = len(unique_hrefs)


   # Scroll and wait for new items to load
    scroll_until_new_items(driver)

    
    # Scroll down to load more products if needed
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #time.sleep(10)  # Adjust the wait time if needed

    # Check if the required number of unique links has been collected
    print(f"Total unique hrefs collected: {len(unique_hrefs)}")
    #print(unique_hrefs)

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
      #  time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)
            product_info[label] = value

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
golden_df.to_csv("golden_scent_perfumes.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


0
No more items to load or page height did not change.
Total unique hrefs collected: 16
16
No more items to load or page height did not change.
Total unique hrefs collected: 16
16
No more items to load or page height did not change.
Total unique hrefs collected: 16
16
No more items to load or page height did not change.
Total unique hrefs collected: 16
16
No more items to load or page height did not change.
Total unique hrefs collected: 16
16
No more items to load or page height did not change.
Total unique hrefs collected: 16
16
Total unique hrefs collected: 16
16
Total unique hrefs collected: 20
Error on perfume 2: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF7B55338A5+3004357]
	(No symbol) [0x00007FF7B51C9970]
	(No symbol) [0x00007FF7B507582A]
	(No symbol) [0x00007FF7B5063C07]
	(No symbol) [0x00007FF7B506

In [19]:
golden_df

NameError: name 'golden_df' is not defined

In [20]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import re  # Import regular expressions

# Set up Edge options to run the browser in headless mode
edge_options = Options()
edge_options.add_argument("--headless")  # Ensure the browser runs headlessly
edge_options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration (optional)
edge_options.add_argument("--no-sandbox")  # Disable the sandbox for security (optional)
edge_options.add_argument("window-size=1200x600")  # Set a fixed window size
edge_options.add_argument("--disable-software-rasterizer")  # For cloud environments

# Initialize the WebDriver with the Edge options
driver = webdriver.Edge(options=edge_options)

# Open the provided URL
driver.get("https://www.goldenscent.com/c/new.html?action=cat&id=243")

# Wait for the page to load
time.sleep(10)  # Adjust the wait time if needed

# Initialize an empty list to store unique hrefs
unique_hrefs = []

# Loop to collect unique hrefs for up to 200 perfumes
while len(unique_hrefs) < 200:
    # Locate the perfume links and extract their href attributes
    golden_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    golden_hrefs = [link.get_attribute('href') for link in golden_links]

    # Collect unique hrefs
    for href in golden_hrefs:
        if href not in unique_hrefs:
            unique_hrefs.append(href)
        if len(unique_hrefs) >= 200:  # Stop once we have 200 unique hrefs
            break

    # Scroll down to load more products if needed
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Adjust the wait time if needed

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)
            product_info[label] = value

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
golden_df.to_csv("golden_scent_perfumes.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


KeyboardInterrupt: 

In [44]:
from bs4 import BeautifulSoup

# Assuming `html_content` contains the HTML as a string
soup = BeautifulSoup(html_content, 'html.parser')

# Select all product links within the 'ais-InfiniteHits-item' class
perfume_links = []
for link in soup.select('.ais-InfiniteHits-item a.product-link'):
    href = link.get('href')
    if href:
        perfume_links.append("https://www.goldenscent.com" + href)

# Display the captured links
print(perfume_links)


[]


In [56]:
men_hrefs=[]
women_hrefs=[]
both_hrefs=[]

In [67]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [85]:
# Function to initialize the driver
def start_driver():
    driver = webdriver.Chrome()  # Or replace with your specific driver
    driver.get(url)
    time.sleep(3)  # Wait for the page to load if necessary
    return driver
    

In [86]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

url="https://www.goldenscent.com/c/fragrances.html?action=cat&id=3"
all_data_dict={}
target_categories = [
    "نسائي",
    "رجالي/نسائي",
    "رجالي",
    "زهرية",
    "شرقية",
    "خشبية",
    "زهرية شرقية",
    "حمضية",
    "فاكهية",
    "أروماتيك",
    "شرقية خشبية",
    "جلدية",
    "عود",
    "حسية",
    "رومانسية",
    "أنثوية",
    "عصرية",
    "جذابة",
    "محبة للطبيعة",
    "متألقة",
    "حيوية",
    "راقية",
    "ديناميكية"
]
                
# Loop through each target category
for target in target_categories:
    
    # Start the driver for each iteration
    driver = start_driver()
    # Find all category elements
    category_elements = driver.find_elements(By.CSS_SELECTOR, ".ais-RefinementList-label")
    
    for category in category_elements:
        label_text = category.find_element(By.CSS_SELECTOR, ".ais-RefinementList-labelText").text.strip()
        
        if label_text == target:
            print(label_text)
            # Click using JavaScript to ensure it registers
            driver.execute_script("arguments[0].click();", category)

            time.sleep(20)

            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                # Scroll down to the bottom of the page
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                # Calculate new scroll height and compare with the last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break  # If heights are the same, break out of the loop
                last_height = new_height
                # Locate the perfume links and extract their href attributes
            golden_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
            golden_hrefs = [link.get_attribute('href') for link in golden_links]

            all_data_dict[target]=golden_hrefs
            time.sleep(4)  # Allow time for the page to load after selection
            break
            driver.quit()
    
print(len(all_data_dict))

    
    
        




# After scrolling, parse the page source with BeautifulSoup
#soup = BeautifulSoup(driver.page_source, 'html.parser')
#driver.quit()

# Base URL for the website
#base_url = "https://www.goldenscent.com"

"""
# Select all product links and compile them into a list
perfume_links = []
for link in soup.select('.ais-InfiniteHits-item a.product-link'):
    href = link.get('href')
    if href:
        perfume_links.append(base_url + href)

# Display the captured links
"""
#women_hrefs=golden_hrefs




نسائي
رجالي/نسائي
رجالي
زهرية
شرقية
خشبية
زهرية شرقية
حمضية
فاكهية
أروماتيك
شرقية خشبية
جلدية
عود
حسية
رومانسية
أنثوية
جذابة
محبة للطبيعة
متألقة
حيوية
راقية
ديناميكية
22


"\n# Select all product links and compile them into a list\nperfume_links = []\nfor link in soup.select('.ais-InfiniteHits-item a.product-link'):\n    href = link.get('href')\n    if href:\n        perfume_links.append(base_url + href)\n\n# Display the captured links\n"

In [71]:
len(golden_hrefs)

1000

In [113]:
print(len(all_data_dict))

23


In [116]:
total=0
# Loop through each category in all_data_dict
for category, links in all_data_dict.items():
    # Count the number of links in the current category
    link_count = len(links)
    total+=link_count
    # Print the category name and the count of links
    print(f"Category: {category}, Link Count: {link_count}")
    print(total)


Category: نسائي, Link Count: 1000
1000
Category: رجالي/نسائي, Link Count: 1000
2000
Category: رجالي, Link Count: 1000
3000
Category: زهرية, Link Count: 1000
4000
Category: شرقية, Link Count: 1000
5000
Category: خشبية, Link Count: 999
5999
Category: زهرية شرقية, Link Count: 751
6750
Category: حمضية, Link Count: 607
7357
Category: فاكهية, Link Count: 539
7896
Category: أروماتيك, Link Count: 584
8480
Category: شرقية خشبية, Link Count: 319
8799
Category: جلدية, Link Count: 278
9077
Category: عود, Link Count: 148
9225
Category: حسية, Link Count: 1000
10225
Category: رومانسية, Link Count: 763
10988
Category: أنثوية, Link Count: 379
11367
Category: جذابة, Link Count: 1000
12367
Category: محبة للطبيعة, Link Count: 1000
13367
Category: متألقة, Link Count: 323
13690
Category: حيوية, Link Count: 1000
14690
Category: راقية, Link Count: 383
15073
Category: ديناميكية, Link Count: 571
15644
Category: كلاسيكية, Link Count: 321
15965


In [111]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

url = "https://www.goldenscent.com/c/fragrances.html?action=cat&id=3"
#all_data_dict = {}

target_categories = [
    "نسائي"
    "رجالي/نسائي",
    "رجالي",
    "زهرية",
    "شرقية",
    "خشبية",
    "زهرية شرقية",
    "حمضية",
    "فاكهية",
    "أروماتيك",
    "شرقية خشبية",
    "جلدية",
    "عود",
    "حسية",
    "رومانسية",
    "أنثوية",  
    "جذابة",
    "محبة للطبيعة",
    "متألقة",
    "حيوية",
    "راقية",
    "ديناميكية",
    "كلاسيكية"
]


# Function to start the driver
def start_driver():
    driver = webdriver.Chrome()  # Make sure ChromeDriver is in your PATH
    driver.get(url)
    time.sleep(3)  # Wait for the page to load
    return driver

# Loop through each target category
for target in target_categories:
    # Start the driver for each iteration
    driver = start_driver()
    time.sleep(17)
    try:
        # Find all category elements
        category_elements = driver.find_elements(By.CSS_SELECTOR, ".ais-RefinementList-label")
        time.sleep(17)
        for category in category_elements:
            #print(category.get_attribute('outerHTML'))
            label_text = category.find_element(By.CSS_SELECTOR, ".ais-RefinementList-labelText").text.strip()
            #print(label_text)
            # Skip the "أطفال" category
            if label_text not in target_categories:
                #print(f"Skipping category: {label_text}")
                continue
                
            if label_text == target:
                print(f"Clicking category: {label_text}")
                
                # Click using JavaScript to ensure it registers
                driver.execute_script("arguments[0].click();", category)
                time.sleep(17)  # Allow time for the page to load after selection

                # Scroll until reaching the bottom of the page
                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(17)  # Pause to allow content to load
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                
                # Locate the perfume links and extract their href attributes
                golden_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
                golden_hrefs = [link.get_attribute('href') for link in golden_links]

                # Store in dictionary
                all_data_dict[target] = golden_hrefs
                print(f"Collected {len(golden_hrefs)} links for category '{label_text}'")
                break  # Move to the next category

    finally:
        driver.quit()  # Ensure the driver closes after each category

print("Total categories processed:", len(all_data_dict))


<label class="ais-RefinementList-label"><span class="ais-RefinementList-labelText">
                                  رجالي/نسائي
                                </span> <span class="ais-RefinementList-count">5783</span></label>
رجالي/نسائي
Skipping category: رجالي/نسائي
<label class="ais-RefinementList-label"><span class="ais-RefinementList-labelText">
                                  نسائي
                                </span> <span class="ais-RefinementList-count">3244</span></label>
نسائي
Skipping category: نسائي
<label class="ais-RefinementList-label"><span class="ais-RefinementList-labelText">
                                  رجالي
                                </span> <span class="ais-RefinementList-count">2057</span></label>
رجالي
Skipping category: رجالي
<label class="ais-RefinementList-label"><span class="ais-RefinementList-labelText">
                                  أطفال
                                </span> <span class="ais-RefinementList-count">89</span></label>

In [115]:
# Combine all lists in `all_data_dict` and remove duplicates
unique_links = list(set(link for links in all_data_dict.values() for link in links))

# `unique_links` now contains all unique links from each list in `all_data_dict`
print(len(unique_links))


8144


In [117]:
unique_links

['https://www.goldenscent.com/p/korloff-cuir-mythique-eau-de-parfum-for-men-and-women.html?action=prod&id=77209',
 'https://www.goldenscent.com/p/calvin-klein-euphoria.html?action=prod&id=748',
 'https://www.goldenscent.com/p/grasse-perfume-oud-the-palace-eau-de-parfum-for-men-women.html?action=prod&id=68223',
 'https://www.goldenscent.com/p/marco-serussi-harmony-edt-for-men.html?action=prod&id=26047',
 'https://www.goldenscent.com/p/lorenzo-villoresi-uomo-perfume-eau-de-toilette-for-men-and-women.html?action=prod&id=8122',
 'https://www.goldenscent.com/p/tory-burch-tory-burch-perfume-eau-de-perfum-for-women.html?action=prod&id=8105',
 'https://www.goldenscent.com/p/epock-genar-eau-de-parfum-for-men.html?action=prod&id=80980',
 'https://www.goldenscent.com/p/khaltat-blends-of-love-salang-eau-de-parfum-for-men-and-women.html?action=prod&id=41068',
 'https://www.goldenscent.com/p/atelier-cologne-clementine-california-eau-de-cologne-for-men-and-women.html?action=prod&id=82644',
 'https://

In [118]:
# Save each URL on a new line in a text file
with open("unique_links.txt", "w", encoding="utf-8") as file:
    for link in unique_links:
        file.write(link + "\n")


In [119]:
import json

# Save all_data_dict as a JSON file
with open("all_data_dict.json", "w", encoding="utf-8") as file:
    json.dump(all_data_dict, file, ensure_ascii=False, indent=4)


In [122]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_links):
    try:
        driver = webdriver.Chrome()  # Or replace with your specific driver
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)
            product_info[label] = value

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")
        driver.quit()
    driver.quit()

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
golden_df.to_csv("golden_scent_perfumes.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


Error on perfume 1865: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF694F038A5+3004357]
	(No symbol) [0x00007FF694B99970]
	(No symbol) [0x00007FF694A4582A]
	(No symbol) [0x00007FF694A42F26]
	(No symbol) [0x00007FF694A33E79]
	(No symbol) [0x00007FF694A35C16]
	(No symbol) [0x00007FF694A3413F]
	(No symbol) [0x00007FF694A339CB]
	(No symbol) [0x00007FF694A3390A]
	(No symbol) [0x00007FF694A3153E]
	(No symbol) [0x00007FF694A31E0C]
	(No symbol) [0x00007FF694A488CA]
	(No symbol) [0x00007FF694ADC86E]
	(No symbol) [0x00007FF694ABBBDA]
	(No symbol) [0x00007FF694ADBA4C]
	(No symbol) [0x00007FF694ABB983]
	(No symbol) [0x00007FF694A87628]
	(No symbol) [0x00007FF694A88791]
	GetHandleVerifier [0x00007FF694F2A00D+3161901]
	GetHandleVerifier [0x00007FF694F7E060+3506048]
	GetHandleVerifier [0x00007FF694F7400D+3465005]
	GetHandleVerifier [0x00007FF694CF0EEB+830987]
	(No symbol) [0x00007FF694BA467F]
	(No symbol) [0x00007FF